In [4]:
function count_information(vector)
    
    mersenne_depth  = log2(length(vector))-1
    alpha           = vector[1]
    max_info        = 0
    actual_info     = 0
    for m in 1:mersenne_depth
        max_info = max_info + 1-(1/(2^m))
    end
    println("max info:\t", max_info)
    for m in 1:mersenne_depth
        first = Int(   (2^(m-1))-1  )
        last  = Int(   (2^ m   )-1  )
        if last == 1
            this_slice = [vector[last]]
            println("\nalpha:\t", alpha)
            deleteat!(vector, last)
            actual_info = actual_info + 1-(1/(2^m))
        else
            this_slice = vector[first:last]
            deleteat!(vector, first:last)
            if this_slice[1] == alpha
                if all(y->y==this_slice[1],this_slice)
                    println("\n identical slice:\t", this_slice)
                    println("SAME AS ALPHA")
                    actual_info = actual_info + 1-(1/(2^m))
                end
            end
        end
    end
    println("actual info:\t", actual_info)
    information_count = rand(Float64, 1)
    return information_count
end

function test_count_information()
    references = [  [[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], "test"],
                    [[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2],                     0.0],                          
                    [[2,2,2,2,1,1,1,1],                     0.0], 
                    [[1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2],     0.0],                          
                    [[2,1,2,1,2,1,2,1,2,1,2,1,2,1],         0.0], 
                    [[2,1,2,1,2,1,2,1,2,1,2,1,2,1],         0.0],                          
                    [[1,2,1,2,1,2,1,2,1,2,1,2,1,1],         0.0], 
                    [[1,1,1,1,1,1,1,1,1,2,2,2,2,2,2,2,2,2], 0.0],                          
                    [[2,2], 0.5],
                    
                    [[1,2,1,2,1,2,1,2],                     0.0],                
                    [[2,1,2,1,2,1,2,1],                     0.0], 
        
                    [[1,2,1,2,1,2,1,2,1,2,1,2,1,2,1,2],     0.0],
                    [[2,1,2,1,2,1,2,1,2,1,2,1,2,1,2,1],     0.0], 
                    [[2,1,1,1,2,2,2,2,1,1,1,1,2,2,2,1],     0.0],
                    [[1,2,2,2,1,1,1,1,2,2,2,2,1,1,1,2],     0.0], 
        
                    [[1,1,2,1,2,1,2,2],                     0.0],                
                    [[2,2,1,2,1,2,1,1],                     0.0], 
                    [[2,1,1,1,1,2,2,2],                     0.0],                  
                    [[1,2,2,2,2,1,1,1],                     0.0], 
                    [[1,2,1,1,2,2],                         0.0],                    
                    [[2,2,1,2,1,1],                         0.0], 
                    [[1,1,1,2,2,2],                         0.0],                      
                    [[2,2,2,1,1,1],                         0.0]
                                                                                                    ]
    for ref in references
        vector = ref[1]
        print("\n\nordered cases in space from origin:\t", vector)
        information_ratio   = ref[2]
        print("\nreference information ratio:\t\t", information_ratio)
        
        information_count = count_information(vector)
        print("\nderived information ratio:\t\t", information_count)
        
        print("\nif we are right then this says TRUE:\t", information_count==information_ratio)
    end
end

test_count_information()




ordered cases in space from origin:	[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
reference information ratio:		testmax info:	2.125

alpha:	1
actual info:	0.5

derived information ratio:		[0.1012932976688643]
if we are right then this says TRUE:	false

ordered cases in space from origin:	[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2]
reference information ratio:		0.0max info:	3.0625

alpha:	1

 identical slice:	[1, 1, 1]
SAME AS ALPHA

 identical slice:	[1, 1, 1, 1, 1]
SAME AS ALPHA

 identical slice:	[1, 1, 1, 1, 1, 1, 1, 1, 1]
SAME AS ALPHA
actual info:	3.0625

derived information ratio:		[0.5900591969687965]
if we are right then this says TRUE:	false

ordered cases in space from origin:	[2, 2, 2, 2, 1, 1, 1, 1]
reference information ratio:		0.0max info:	1.25

alpha:	2

 identical slice:	[2, 2, 2]
SAME AS ALPHA
actual info:	1.25

derived information ratio:		[0.08841477539